In [7]:
import pandas as pd
import numpy as np
from ast import literal_eval
import re
import glob, os
import requests
from bs4 import BeautifulSoup
from DataCleaning import get_nyt_review_text

In [10]:
omdb_files = glob.glob(os.path.join(r'data', "OMDBData*.csv"))
nyt_files = glob.glob(os.path.join(r'data', "NYTData*.csv"))

omdb_df_temp = (pd.read_csv(f) for f in omdb_files)
nyt_df_temp = (pd.read_csv(f) for f in nyt_files)
omdb_df   = pd.concat(omdb_df_temp, ignore_index=True)
nyt_df   = pd.concat(nyt_df_temp, ignore_index=True)
omdb_df = omdb_df.loc[:, ~omdb_df.columns.str.contains('^Unnamed')]
nyt_df = nyt_df.loc[:, ~nyt_df.columns.str.contains('^Unnamed')]

In [ ]:
# Clean OMDB Ratings
omdb_df["Ratings"]=omdb_df["Ratings"].apply(literal_eval).apply(lambda x: dict([list(d.values()) for d in x]))
unpacked_ratings_df=omdb_df["Ratings"].apply(pd.Series)
omdb_df=omdb_df.join(unpacked_ratings_df)
omdb_df["Internet Movie Database"]=omdb_df["Internet Movie Database"].str.extract(r"(.*?)\/").astype(float)
omdb_df["Rotten Tomatoes"]= omdb_df["Rotten Tomatoes"].str.extract(r"(.*?)\%").astype(float)
omdb_df["Metacritic"]=omdb_df["Metacritic"].str.extract(r"(.*?)\/").astype(float)

In [ ]:
# Scrape NYT movie reviews for full text:
nyt_df['review_text']=nyt_df['link'].apply(lambda x: get_nyt_review_text(literal_eval(x)['url']))
nyt_df.to_csv("data/NYTData_wReviewText.csv",encoding='utf-8-sig')